In [1]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
import torch.nn.functional as F
from torch_geometric.utils.convert import from_scipy_sparse_matrix, from_networkx
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm
import torch.nn as nn


import random
import numpy as np
from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip
from scipy import sparse
from torch.utils.data import random_split
import time

# load pickle module
import pickle
import networkx as nx
# from tqdm import tqdm
import sys
import h5py

In [2]:
class GCN(torch.nn.Module):  # the simpliest model that GNN and it is classical, used as baseline
    def __init__(self, num_node_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 128)
        self.conv2 = GCNConv(128, 64)
        self.conv3 = GCNConv(64, 16)
        self.conv4 = GCNConv(16, 8)
        self.fc = torch.nn.Linear(8, 1)

    """
        hyperparameters:
        - number of hidden layers
        - number of hidden channels
        - dropout rate (now it's zero)
        - learning rate <- most important to tune
        - weight decay
        - etc etc.
    """

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = self.conv1(x, edge_index, edge_weight)
        x = F.elu(x)

        x = self.conv2(x, edge_index, edge_weight)
        x = F.elu(x)

        x = self.conv3(x, edge_index, edge_weight)
        x = F.elu(x)

        x = self.conv4(x, edge_index, edge_weight)
        x = F.elu(x)

        x = global_mean_pool(x, batch)
        x = self.fc(x)

        return x

In [3]:
def train(model, myloader, optimizer, device):
    model.train()
    loss_all = 0
    correct = 0
    y_true = []
    total = 0
    results = []
    for data in myloader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        y_true.append(label)
        loss = F.mse_loss(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()

        optimizer.step()
        results.append(output)

    return loss_all / len(myloader.dataset), results, y_true

In [4]:
def validate(model, loader, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # No need to track gradients during validation
        for data in loader:
            data = data.to(device)
            out = model(data)
            # Assume you have a loss function defined, e.g., MSE for regression
            loss = F.mse_loss(out, data.y)
            total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

In [5]:
def test(model, testloader, device):
    model.eval()
    predictions = []
    total_loss = 0
    true_values = []
    with torch.no_grad():
        for data in testloader:
            data = data.to(device)
            output = model(data)
            loss = F.mse_loss(output, data.y)
            total_loss += loss.item() * data.num_graphs
            predictions.append(output.cpu())
            true_values.append(data.y.cpu())
    return torch.cat(predictions, dim=0), torch.cat(true_values, dim=0), total_loss / len(testloader.dataset)

In [6]:
num_x = 7

In [8]:
es_idx = 5

In [38]:
torch.manual_seed(1)
all_data_len = len(dataset)
train_size = int(all_data_len * 0.6)
val_size = int(all_data_len * 0.2)
test_size = all_data_len - train_size - val_size

# train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
train_data1, val_data1, test_data1 = random_split(dataset, [train_size, val_size, test_size])

In [36]:
%run ../../codes/statistics.py

In [39]:
compare_datasets(train_data1, train_data)

False

In [35]:
train_data1[0]

Data(x=[3142, 7], edge_index=[2, 10378], edge_attr=[10378, 1], y=[1, 1])

In [18]:
# dataset = torch.load('/Users/qingyao/Documents/branching_data/gnn_regression/dataset_7.pt')

#     # WN = np.loadtxt('W_avg.csv')
# lr_list = [np.power(0.5, i) for i in range(2, 16, 2)]*10
# my_lr = lr_list[es_idx]



    # Now we can create a DataLoader
# train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
# val_loader = DataLoader(val_data, batch_size=128, shuffle=False)
# Create a model and an optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_node_features=num_x).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=my_lr, weight_decay=5e-4)

# training and validation
counter = 0
count_epochs = 0
best = float("inf")
epochs = 100
patience = 10
loss_ep = []
# define the random seed

seed = 1 #int(time.time()) + es_idx
# random.seed(seed)
torch.manual_seed(seed)
    # random_float = random.uniform(0, 1)
    # torch.manual_seed(es_idx)
print('start training')
for epoch in range(epochs):
    loss, myres, reals = train(model, train_loader, optimizer, device)
    # loss_ep.append(loss)
    val_loss = validate(model, val_loader, device)
    loss_ep.append(val_loss)
    print(epoch,val_loss)
    if val_loss < best:
        best = val_loss
        counter = 0
#         torch.save(model.state_dict(), '/rds/general/user/qy1815/home/branching_superspreading/regression_{}/best_model_{}_{}.pth'.format(
#             num_x, s, es_idx))  # Save the best model
    else:
        counter += 1
        count_epochs += 1

    if counter > patience:
        break

# testing
# test_loader = DataLoader(test_data, batch_size=128, shuffle=True)
# predictions, y_true, test_mse = test(model, test_loader, device)

# predictions_np = predictions.cpu().detach().numpy()
# y_true_np = y_true.cpu().detach().numpy()

# with h5py.File('/rds/general/user/qy1815/home/branching_superspreading/regression_{}/res_{}_{}.hdf5'.format(num_x, s, es_idx), 'w') as f:
#     # f['val_mse'] = val_loss
#     f.create_dataset('val_mse', data=np.array(loss_ep))
#     f.create_dataset('predictions', data=predictions_np)
#     f.create_dataset('y_true', data=y_true_np)
#     f['test_mse'] = test_mse


start training
0 4.119335630204943


KeyboardInterrupt: 

In [ ]:
loss_ep

In [ ]:
start training
0 4.239625602298313